## Work
1. 請自行定義一個 loss function, 為 0.3 * focal loss + 0.7 cross-entropy，訓練並比較結果


In [1]:
import os
import keras

# Disable GPU
os.environ["CUDA_VISIBLE_DEVICES"] = ""

Using TensorFlow backend.


In [2]:
train, test = keras.datasets.cifar10.load_data()

In [3]:
## 資料前處理
def preproc_x(x, flatten=True):
    x = x / 255.
    if flatten:
        x = x.reshape((len(x), -1))
    return x

def preproc_y(y, num_classes=10):
    if y.shape[-1] == 1:
        y = keras.utils.to_categorical(y, num_classes)
    return y    

In [4]:
x_train, y_train = train
x_test, y_test = test

# Preproc the inputs
x_train = preproc_x(x_train)
x_test = preproc_x(x_test)

# Preprc the outputs
y_train = preproc_y(y_train)
y_test = preproc_y(y_test)

In [5]:
from keras.layers import BatchNormalization
def build_mlp(input_shape, output_units=10, num_neurons=[512, 256, 128]):
    """Code Here
    建立你的神經網路
    """
def build_mlp(input_shape, output_units=10, num_neurons=[512, 256, 128]):
    input_layer = keras.layers.Input(input_shape)
    
    for i, n_units in enumerate(num_neurons):
        if i == 0:
            x = keras.layers.Dense(units=n_units, 
                                   activation="relu", 
                                   name="hidden_layer"+str(i+1))(input_layer)
            x = BatchNormalization()(x)
        else:
            x = keras.layers.Dense(units=n_units, 
                                   activation="relu", 
                                   name="hidden_layer"+str(i+1))(x)
            x = BatchNormalization()(x)
    
    out = keras.layers.Dense(units=output_units, activation="softmax", name="output")(x)
    
    model = keras.models.Model(inputs=[input_layer], outputs=[out])
    return model

In [6]:
## 超參數設定
LEARNING_RATE = 1e-3
EPOCHS = 25
BATCH_SIZE = 1024
MOMENTUM = 0.95

In [22]:
import tensorflow as tf
import keras.backend as K

"""Code Here
撰寫一個 loss function, 使其可以結合 focal loss 與 crossentropy loss
"""
def combined_loss(gamma=2., alpha=4., ce_weights=0.7, fcl_weights=0.3):
    gamma = float(gamma)
    alpha = float(alpha)
    def combined_loss_fixed(y_true, y_pred):
        epsilon = 1e-8
        y_true = tf.convert_to_tensor(y_true, tf.float32)
        y_pred = tf.convert_to_tensor(y_pred, tf.float32)
        
        model_out = tf.add(y_pred, epsilon)
        ce = tf.multiply(y_true, -tf.math.log(model_out))
        weight = tf.multiply(y_true, tf.pow(tf.subtract(1., model_out), gamma))
        fl = tf.multiply(alpha, tf.multiply(weight, ce))
        reduced_fl = tf.reduce_max(fl, axis=1)
        
        ce_loss = keras.losses.categorical_crossentropy(y_true, y_pred)
        
        return ce_loss * ce_weights + tf.reduce_mean(reduced_fl) * fcl_weights
    return combined_loss_fixed

ce_weights_list = [0., 0.3, 0.5, 0.7, 1]

In [24]:
import itertools
results = {}

for i, ce_w in enumerate(ce_weights_list):
    print("Numbers of exp: %i, ce_weight: %.2f" % (i, ce_w))

    model = build_mlp(input_shape=x_train.shape[1:])
    model.summary()
    optimizer = keras.optimizers.SGD(lr=LEARNING_RATE, nesterov=True, momentum=MOMENTUM)
    """Code Here
    將自定義的 loss function 加入模型
    """
    model.compile(loss=combined_loss(ce_weights=ce_w, fcl_weights=1-ce_w), metrics=["accuracy"], optimizer=optimizer)

    model.fit(x_train, y_train, 
              epochs=EPOCHS, 
              batch_size=BATCH_SIZE, 
              validation_data=(x_test, y_test), 
              shuffle=True
             )
    
    # Collect results
    exp_name_tag = ("exp-%s" % (i))
    results[exp_name_tag] = {'train-loss': model.history.history["loss"],
                             'valid-loss': model.history.history["val_loss"],
                             'train-acc': model.history.history["accuracy"],
                             'valid-acc': model.history.history["val_accuracy"]}

Numbers of exp: 0, ce_weight: 0.00
Model: "model_8"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_8 (InputLayer)         (None, 3072)              0         
_________________________________________________________________
hidden_layer1 (Dense)        (None, 512)               1573376   
_________________________________________________________________
batch_normalization_22 (Batc (None, 512)               2048      
_________________________________________________________________
hidden_layer2 (Dense)        (None, 256)               131328    
_________________________________________________________________
batch_normalization_23 (Batc (None, 256)               1024      
_________________________________________________________________
hidden_layer3 (Dense)        (None, 128)               32896     
_________________________________________________________________
batch_normalization_24 (

Epoch 11/25
50000/50000 [==============================] - 8s 166us/step - loss: 2.2325 - accuracy: 0.6183 - val_loss: 3.2289 - val_accuracy: 0.4764
Epoch 12/25
50000/50000 [==============================] - 8s 166us/step - loss: 2.1337 - accuracy: 0.6325 - val_loss: 3.1471 - val_accuracy: 0.4904
Epoch 13/25
50000/50000 [==============================] - 9s 179us/step - loss: 2.0400 - accuracy: 0.6461 - val_loss: 3.1910 - val_accuracy: 0.4900
Epoch 14/25
50000/50000 [==============================] - 9s 174us/step - loss: 1.9475 - accuracy: 0.6593 - val_loss: 3.1957 - val_accuracy: 0.4916
Epoch 15/25
50000/50000 [==============================] - 8s 169us/step - loss: 1.8607 - accuracy: 0.6713 - val_loss: 3.2190 - val_accuracy: 0.4870
Epoch 16/25
50000/50000 [==============================] - 8s 169us/step - loss: 1.7791 - accuracy: 0.6841 - val_loss: 3.2974 - val_accuracy: 0.4785
Epoch 17/25
50000/50000 [==============================] - 9s 187us/step - loss: 1.6901 - accuracy: 0.6978

Train on 50000 samples, validate on 10000 samples
Epoch 1/25
50000/50000 [==============================] - 9s 188us/step - loss: 3.4213 - accuracy: 0.3045 - val_loss: 3.4277 - val_accuracy: 0.2141
Epoch 2/25
50000/50000 [==============================] - 8s 167us/step - loss: 2.5798 - accuracy: 0.4189 - val_loss: 3.0701 - val_accuracy: 0.3129
Epoch 3/25
50000/50000 [==============================] - 9s 183us/step - loss: 2.3617 - accuracy: 0.4597 - val_loss: 2.7542 - val_accuracy: 0.3916
Epoch 4/25
50000/50000 [==============================] - 8s 167us/step - loss: 2.2269 - accuracy: 0.4868 - val_loss: 2.5945 - val_accuracy: 0.4086
Epoch 5/25
50000/50000 [==============================] - 8s 169us/step - loss: 2.1294 - accuracy: 0.5060 - val_loss: 2.4457 - val_accuracy: 0.4387
Epoch 6/25
50000/50000 [==============================] - 8s 166us/step - loss: 2.0458 - accuracy: 0.5235 - val_loss: 2.3380 - val_accuracy: 0.4540
Epoch 7/25
50000/50000 [==============================] - 9s 1

50000/50000 [==============================] - 8s 167us/step - loss: 1.0692 - accuracy: 0.6381 - val_loss: 1.4549 - val_accuracy: 0.4906
Epoch 21/25
50000/50000 [==============================] - 9s 173us/step - loss: 1.0473 - accuracy: 0.6450 - val_loss: 1.4582 - val_accuracy: 0.4940
Epoch 22/25
50000/50000 [==============================] - 8s 167us/step - loss: 1.0250 - accuracy: 0.6536 - val_loss: 1.4628 - val_accuracy: 0.4905
Epoch 23/25
50000/50000 [==============================] - 8s 169us/step - loss: 1.0033 - accuracy: 0.6635 - val_loss: 1.4561 - val_accuracy: 0.4932
Epoch 24/25
50000/50000 [==============================] - 9s 170us/step - loss: 0.9821 - accuracy: 0.6712 - val_loss: 1.4628 - val_accuracy: 0.4925
Epoch 25/25
50000/50000 [==============================] - 9s 177us/step - loss: 0.9621 - accuracy: 0.6788 - val_loss: 1.4661 - val_accuracy: 0.4912


In [ ]:
import matplotlib.pyplot as plt
import matplotlib.cm as mplcm
import matplotlib.colors as colors
%matplotlib inline
NUM_COLORS = len(results.keys())

cm = plt.get_cmap('gist_rainbow')
cNorm  = colors.Normalize(vmin=0, vmax=NUM_COLORS-1)
scalarMap = mplcm.ScalarMappable(norm=cNorm, cmap=cm)
color_bar = [scalarMap.to_rgba(i) for i in range(NUM_COLORS)]

plt.figure(figsize=(8,6))
for i, cond in enumerate(results.keys()):
    plt.plot(range(len(results[cond]['train-loss'])),results[cond]['train-loss'], '-', label=cond, color=color_bar[i])
    plt.plot(range(len(results[cond]['valid-loss'])),results[cond]['valid-loss'], '--', label=cond, color=color_bar[i])
plt.title("Loss")
plt.legend(loc='center left', bbox_to_anchor=(1, 0.5))
plt.show()

plt.figure(figsize=(8,6))
for i, cond in enumerate(results.keys()):
    plt.plot(range(len(results[cond]['train-acc'])),results[cond]['train-acc'], '-', label=cond, color=color_bar[i])
    plt.plot(range(len(results[cond]['valid-acc'])),results[cond]['valid-acc'], '--', label=cond, color=color_bar[i])
plt.title("Accuracy")
plt.legend(loc='center left', bbox_to_anchor=(1, 0.5))
plt.show()